In [65]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier


In [66]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [67]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Columns: 555 entries, Unnamed: 0 to nk_feat_13
dtypes: float64(554), int64(1)
memory usage: 21.7 MB


In [68]:
selector = SelectKBest(score_func=f_classif, k=300)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

In [69]:
#x_train= x_train.iloc[:,cols]
#x_test = x_test.iloc[:,cols]

In [70]:
#feature reduction with pca
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
pca.fit(x_train)
#x_train = pca.transform(x_train)
#x_test = pca.transform(x_test)

PCA(n_components=200)

In [71]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [72]:
import xgboost as xgb
clf = xgb.XGBClassifier(n_estimators=100,objective="multi:softmax", random_state=42)

In [73]:
#cross validation score
scores = cross_val_score(clf, x_train, np.ravel(y_train), cv=10, scoring='f1_micro', n_jobs=-1)
print(scores.mean())
print(scores.std())
#0.6994274400684931 StandardScaler
#0.6996231347847358 / 0.01592645870348861  MinMaxScaler
#0.6818420529598825 PCA 100
#0.6794921875 PCA 100


0.6996231347847358
0.01592645870348861


In [74]:
#save the model with pickle
import pickle
filename = 'models/xgboost_model.sav'
pickle.dump(clf, open(filename, 'wb'))
